In [1]:
using Pkg
Pkg.activate("/Users/jiyongso/.juliaenv/juliaNumerical")
using ForwardDiff


  Activating project at `~/.juliaenv/juliaNumerical`


In [2]:
g(x::Vector) = x[1]+ 2*x[2]
ForwardDiff.gradient(g, [1.0, 1.0])

2-element Vector{Float64}:
 1.0
 2.0

In [3]:
f(x::Real) = x*sin(x)
ForwardDiff.derivative(f, π)

-3.141592653589793

In [5]:
H(x::Vector) = [x[1]*x[2], x[1]+x[2]]
ForwardDiff.jacobian(H, [1.0, 2.0])

2×2 Matrix{Float64}:
 2.0  1.0
 1.0  1.0

In [6]:
struct DNumber{T} 
    v::T
    e::T

    function DNumber(v::Real, e::Union{Real, Nothing}=nothing)
        if e == nothing
            e=one(v)
            new{T}(v, e)
        else
            T = promote_type(typeof(v), typeof(e))
            new{T}(v, e)
        end
    end

    
end

In [7]:
Base.:+(x::DNumber, y::DNumber) = DNumber(x.v + y.v, x.e + y.e)
Base.:-(x::DNumber, y::DNumber) = DNumber(x.v - y.v, x.e - y.e)
Base.:*(x::DNumber, y::DNumber) = DNumber(x.v * y.v, x.e * y.v + x.v * y.e)
Base.:/(x::DNumber, y::DNumber) = DNumber(x.v / y.v, (x.e * y.v - x.v * y.e) / y.v^2)
Base.:-(x::DNumber) = DNumber(-x.v, -x.e)
Base.:+(x::DNumber, y::Real) = DNumber(x.v + y, x.e)
Base.:+(x::Real, y::Real) = y+x
Base.:-(x::DNumber, y::Real) = DNumber(x.v-y, x.e)
Base.:-(x::Real, y::DNumber) = -(y-x)
Base.:*(x::DNumber, y::Number) = x*DNumber(y, zero(y))
Base.:*(x::Number, y::DNumber) = DNumber(x, zero(x))*y
Base.:/(x::DNumber, y::Number) = x/DNumber(y, zero(y))
Base.:/(x::Number, y::DNumber) = DNumber(x, zero(x))/y
Base.:^(x::DNumber, y::Real) = DNumber((x.v)^y, y*((x.v)^(y-1))*x.e)

import Base.sin, Base.cos, Base.tan, Base.exp, Base.log
sin(x::DNumber) = DNumber(sin(x.v), cos(x.v)*x.e)
cos(x::DNumber) = DNumber(cos(x.v), -sin(x.v)*x.e)  
tan(x::DNumber) = sin(x)/cos(x)
exp(x::DNumber) = DNumber(exp(x.v), exp(x.v)*x.e)
log(x::DNumber) = DNumber(log(x.v), x.e/x.v)



log (generic function with 42 methods)

In [ ]:
auto_diff(f::Function, v::Real) = f(DNumber(v, one(v))).e

autodiff (generic function with 1 method)

In [11]:
DNumber(2.0, 1.0)^5

DNumber{Float64}(32.0, 80.0)

In [12]:
autodiff(x->(sin(x))^3, π/3)

1.125

In [14]:
3*cos(π/3)*(sin(π/3))^2

1.1250000000000002